### QSP Hadamard Approximation

**Aryan Bawa**  

In this notebook, I modify Pennylane.ai's "Function Fitting using Quantum Signal Processing" tutorial [here](https://pennylane.ai/qml/demos/function_fitting_qsp) such that I can approximate the action of a Hadamard gate. There are a few main things I change:

1. Outputting the full matrix for the circuit, not just the polynomial $ \langle 0 | U_{QSP}|0 \rangle $
2. Changing how we measure how accurate the approximation is by using Fidelity.
3.
4. 

In [ ]:
# have to do this because some libraries using OpenMP cause issues with multiple threads
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import torch
import pennylane as qml
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# defining important functions
def rotation_mat(a):
    """Given a fixed value 'a', compute the signal rotation matrix W(a).
    (requires -1 <= 'a' <= 1)
    """
    diag = a
    off_diag = (1 - a**2) ** (1 / 2) * 1j
    W = [[diag, off_diag], [off_diag, diag]]

    return W

def generate_many_sro(a_vals):
    """Given a tensor of possible 'a' vals, return a tensor of W(a)"""
    w_array = []
    for a in a_vals:
        w = rotation_mat(a)
        w_array.append(w)

    return torch.tensor(w_array, dtype=torch.complex64, requires_grad=False)

def target_hadamard():
    """Return the target Hadamard matrix"""
    return torch.tensor([[1/np.sqrt(2), 1/np.sqrt(2)], 
                        [1/np.sqrt(2), -1/np.sqrt(2)]], 
                       dtype=torch.complex64)

# fidelity-based loss function for the QSP unitary fitting
# I use Pennylane's built-in fidelity function, but I have to convert the tensors to numpy arrays first, and at the end I convert the result back to a torch tensor
def fidelity_loss(pred_unitaries, target_op):
    batch_size = pred_unitaries.shape[0]
    fidelities = []
    
    # Convert target_op to numpy for PennyLane compatibility
    target_np = target_op.detach().numpy() if hasattr(target_op, 'detach') else target_op.numpy()
    
    for i in range(batch_size):
        U_pred = pred_unitaries[i]
        U_pred_np = U_pred.detach().numpy() if hasattr(U_pred, 'detach') else U_pred.numpy() # convert to numpy
        fidelity = qml.math.fidelity(U_pred_np, target_np, check_state=False, c_dtype="complex128")
        fidelities.append(fidelity)
    
    avg_fidelity = torch.mean(torch.tensor(fidelities, dtype=torch.float32)) # convert to torch tensor
    return 1.0 - avg_fidelity
    

# below is the QSP circuit itself - we don't change the basis this time since we don't need to use the relaxed third condition for QSP
@qml.qnode(qml.device("default.qubit", wires=1))
def QSP_circ(phi, W):
    for angle in phi[:-1]:
        qml.RZ(angle, wires=0)
        qml.QubitUnitary(W, wires=0)
    qml.RZ(phi[-1], wires=0)
    return

The Pennylane article uses machine learning to fit specific $ \phi $ values such that the polynomial in $ \langle 0 | U_{QSP}|0 \rangle $ is approximately equal to some P(a). This is done by creating several classes for fitting and running the model. Starting with the fitting class:

In [ ]:
torch_pi = torch.Tensor([math.pi])

class QSP_Unitary_Fit(torch.nn.Module):
    def __init__(self, degree, random_seed=None):
        """Given the degree and number of samples, this method randomly
        initializes the parameter vector (randomness can be set by random_seed)
        """
        super().__init__()
        if random_seed is None:
            self.phi = torch_pi * torch.rand(degree + 1, requires_grad=True)

        else:
            gen = torch.Generator()
            gen.manual_seed(random_seed)
            self.phi = torch_pi * torch.rand(degree + 1, requires_grad=True, generator=gen)

        self.phi = torch.nn.Parameter(self.phi)
        self.num_phi = degree + 1

    def forward(self, omega_mats):
        """PennyLane forward implementation"""
        unitary_pred = []
        generate_qsp_mat = qml.matrix(QSP_circ, wire_order=[0])

        for w in omega_mats:
            u_qsp = generate_qsp_mat(self.phi, w)
            unitary_pred.append(u_qsp)

        return torch.stack(unitary_pred, 0)

Next, the Model_Runner class handles running the optimization, storing the results, determining fidelity between approximate and target states, and plotting this metric.

In [ ]:
# cleaned this up and added verbose output with Claude AI's help
class Model_Runner:
    def __init__(self, degree, num_samples):
        self.degree = degree
        self.num_samples = num_samples
        # generating a values and corresponding W(a) matrices
        self.a_vals = torch.linspace(-0.99, 0.99, num_samples)
        self.omega_mats = generate_many_sro(self.a_vals)
        self.target_op = target_hadamard()

        print(f"Initialized model runner")

    def execute(self, max_iterations=5000, lr=1e-3, random_seed=42, verbose=True):
        # starting up the model
        self.model = QSP_Unitary_Fit(self.degree, random_seed=random_seed)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        
        # storing the initial for comparison
        with torch.no_grad():
            self.init_pred = self.model(self.omega_mats)
        
        # Training tracking
        self.losses = []
        self.fidelities = []
        
        if verbose:
            print(f"\nTraining QSP to approximate Hadamard gate...")
            print(f"Learning rate: {lr}")
            print("-" * 50)
        
        for t in range(max_iterations):
            pred_unitaries = self.model(self.omega_mats)
            loss = fidelity_loss(pred_unitaries, self.target_op)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            current_loss = loss.item()
            current_fidelity = 1.0 - current_loss
            
            self.losses.append(current_loss)
            self.fidelities.append(current_fidelity)
            
            if verbose and (t % 500 == 0 or t < 10):
                print(f"Iteration {t:5d}: Loss = {current_loss:.8f}, Avg Fidelity = {current_fidelity:.8f}")
            
            # Early stopping for very high fidelity
            if current_fidelity > 0.9999:
                if verbose:
                    print(f"High fidelity achieved! Stopping at iteration {t}")
                break
            
            # Early stopping for convergence
            if current_loss < 1e-8:
                if verbose:
                    print(f"Converged at iteration {t}")
                break
        
        # Store final results
        self.final_loss = current_loss
        self.final_fidelity = current_fidelity
        self.training_iterations = len(self.losses)
        
        if verbose:
            print(f"\nTraining completed after {self.training_iterations} iterations")
            print(f"Final loss: {self.final_loss:.8f}")
            print(f"Final average fidelity: {self.final_fidelity:.8f}")

    def evaluate(self, verbose=True):
        """
        Evaluate the quality of the approximation
        """
        if not hasattr(self, 'model'):
            raise ValueError("Must run execute() before evaluate()")
        
        with torch.no_grad():
            self.final_pred = self.model(self.omega_mats)
        
        # Compute fidelities and errors for each 'a' value
        self.fidelities_per_a = []
        self.errors_per_a = []
        
        for i, a in enumerate(self.a_vals):
            U_pred = self.final_pred[i]
            
            # Fidelity using PennyLane's built-in function
            U_pred_np = U_pred.detach().numpy()
            target_np = self.target_op.detach().numpy()
            fidelity = qml.math.fidelity(U_pred_np, target_np, check_state=False, c_dtype="complex128")
            self.fidelities_per_a.append(fidelity.item() if hasattr(fidelity, 'item') else float(fidelity))
            
            # Frobenius error for additional analysis
            diff = U_pred - self.target_op
            frobenius_error = torch.sqrt(torch.sum(torch.abs(diff)**2)).item()
            self.errors_per_a.append(frobenius_error)
        
        # Compute statistics
        self.avg_fidelity = np.mean(self.fidelities_per_a)
        self.min_fidelity = np.min(self.fidelities_per_a)
        self.max_fidelity = np.max(self.fidelities_per_a)
        self.std_fidelity = np.std(self.fidelities_per_a)
        self.avg_error = np.mean(self.errors_per_a)
        
        if verbose:
            print(f"\n=== Hadamard Gate Approximation Quality ===")
            print(f"Average Fidelity:     {self.avg_fidelity:.8f}")
            print(f"Minimum Fidelity:     {self.min_fidelity:.8f}")
            print(f"Maximum Fidelity:     {self.max_fidelity:.8f}")
            print(f"Fidelity Std Dev:     {self.std_fidelity:.8f}")
            print(f"Avg Frobenius Error:  {self.avg_error:.8f}")

    def plot_results(self, show=True, figsize=(15, 5)):
        """
        Plot comprehensive training and evaluation results
        """
        if not hasattr(self, 'losses'):
            raise ValueError("Must run execute() before plotting")
        
        if not hasattr(self, 'fidelities_per_a'):
            self.evaluate(verbose=False)
        
        plt.figure(figsize=figsize)
        
        # Plot 1: Training progress
        plt.subplot(1, 3, 1)
        plt.plot(self.losses, label='Loss', color='red', alpha=0.7)
        plt.plot(self.fidelities, label='Avg Fidelity', color='blue', alpha=0.7)
        plt.xlabel('Iteration')
        plt.ylabel('Value')
        plt.title('Training Progress')
        plt.legend()
        plt.yscale('log')
        plt.grid(True)
        
        # Plot 2: Fidelity vs parameter 'a'
        plt.subplot(1, 3, 2)
        plt.plot(self.a_vals.numpy(), self.fidelities_per_a, 'o-', markersize=4, color='blue', label='Per-sample fidelity')
        plt.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Perfect Fidelity')
        plt.axhline(y=self.avg_fidelity, color='red', linestyle='-', alpha=0.7, 
                    label=f'Average: {self.avg_fidelity:.6f}')
        plt.xlabel('Parameter a')
        plt.ylabel('Fidelity')
        plt.title('Fidelity vs Parameter a')
        plt.legend()
        plt.grid(True)
        plt.ylim([max(0.95, self.min_fidelity - 0.01), 1.002])
        
        # Plot 3: Error analysis
        plt.subplot(1, 3, 3)
        plt.plot(self.a_vals.numpy(), self.errors_per_a, 'o-', markersize=4, color='red', label='Frobenius error')
        plt.axhline(y=self.avg_error, color='blue', linestyle='-', alpha=0.7,
                    label=f'Average: {self.avg_error:.6f}')
        plt.xlabel('Parameter a')
        plt.ylabel('Frobenius Error')
        plt.title('Approximation Error vs Parameter a')
        plt.legend()
        plt.grid(True)
        plt.yscale('log')
        
        plt.tight_layout()
        
        if show:
            plt.show()

    def get_learned_parameters(self):
        """
        Get the learned φ parameters
        """
        if not hasattr(self, 'model'):
            raise ValueError("Must run execute() before getting parameters")
        return self.model.phi.detach().numpy()

    def compare_matrices(self, test_a=0.5, verbose=True):
        """
        Compare target vs approximated matrix for a specific 'a' value
        """
        if not hasattr(self, 'model'):
            raise ValueError("Must run execute() before matrix comparison")
        
        test_W = torch.tensor(rotation_mat(test_a), dtype=torch.complex64)
        
        with torch.no_grad():
            generate_qsp_mat = qml.matrix(QSP_circ_unitary, wire_order=[0])
            approx_H = generate_qsp_mat(self.model.phi, test_W)
            
        if verbose:
            print(f"\n=== Matrix Comparison (a = {test_a}) ===")
            print("Target Hadamard:")
            print(self.target_op.numpy())
            print("\nApproximated Hadamard:")
            print(approx_H.numpy())
            
            fidelity = qml.math.fidelity(approx_H.numpy(), self.target_op.numpy(), check_state=False)
            print(f"\nMatrix fidelity at a={test_a}: {fidelity:.8f}")
        
        return approx_H, fidelity

Finally, we can actually run the model to approximate a Hadamard:

In [ ]:
# Initialize the runner
runner = Model_Runner(degree=9, num_samples=30)

# Execute training
runner.execute(max_iterations=5000, lr=1e-3, random_seed=42)

# Evaluate the results
runner.evaluate()

# Plot comprehensive results
runner.plot_results()

# Display learned parameters and additional analysis
print(f"\n=== Final Results ===")
learned_params = runner.get_learned_parameters()
print(f"Learned φ parameters: {learned_params}")
print(f"Parameter range: [{learned_params.min():.4f}, {learned_params.max():.4f}]")
print(f"Training iterations: {runner.training_iterations}")

# Show matrix comparison for a specific 'a' value
runner.compare_matrices(test_a=0.5)

# Optional: Compare different configurations
print(f"\n{'='*60}")
print("=== Comparing Different Configurations ===")

configs = [
    {"degree": 7, "samples": 20, "lr": 1e-3},
    {"degree": 9, "samples": 25, "lr": 5e-4},
    {"degree": 11, "samples": 30, "lr": 1e-4}
]

results = []

for i, config in enumerate(configs):
    print(f"\nConfiguration {i+1}: {config}")
    runner_test = Model_Runner(config["degree"], config["samples"])
    runner_test.execute(max_iterations=2000, lr=config["lr"], verbose=False)
    runner_test.evaluate(verbose=False)
    
    results.append({
        'config': config,
        'final_fidelity': runner_test.final_fidelity,
        'avg_fidelity': runner_test.avg_fidelity,
        'std_fidelity': runner_test.std_fidelity,
        'iterations': runner_test.training_iterations
    })
    
    print(f"  Final training fidelity: {runner_test.final_fidelity:.6f}")
    print(f"  Average evaluation fidelity: {runner_test.avg_fidelity:.6f}")
    print(f"  Fidelity std dev: {runner_test.std_fidelity:.6f}")
    print(f"  Training iterations: {runner_test.training_iterations}")

# Plot configuration comparison
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
degrees = [r['config']['degree'] for r in results]
avg_fidelities = [r['avg_fidelity'] for r in results]
std_fidelities = [r['std_fidelity'] for r in results]

plt.errorbar(degrees, avg_fidelities, yerr=std_fidelities, 
                marker='o', capsize=5, capthick=2)
plt.xlabel('Polynomial Degree')
plt.ylabel('Average Fidelity')
plt.title('Fidelity vs Polynomial Degree')
plt.grid(True)

plt.subplot(1, 2, 2)
samples = [r['config']['samples'] for r in results]
iterations = [r['iterations'] for r in results]

plt.scatter(samples, iterations, c=degrees, cmap='viridis', s=100)
plt.colorbar(label='Degree')
plt.xlabel('Number of Samples')
plt.ylabel('Training Iterations')
plt.title('Training Efficiency')
plt.grid(True)

plt.tight_layout()
plt.show()